**Datasets Notes on Range/Resolution**

1. Daymet v4 (Meterologic Data)
    * North America
    * 1980.01.01 to 2023.12.31
    * 1000m x 1000m, Daily
    * EEID: NASA/ORNL/DAYMET_V4

2. MODIS (Terra 1km) - EVI/NDVI
    * Global
    * 2000.02.18 to 2024.10.31
    * 1000m x 1000m, 16-day Composite
    * EEID: MODIS/061/MOD13A2

3. MODIS (Terra 500m) - ET/LE
    * Global
    * 2000.01.01 to 2023.12.27
    * 500m x 500m, 8-Day composite
    * MODIS/061/MOD16A2GF

4. MODIS (Terra 500m) - Leaf Area and FPAR
    * Global
    * 2000.02.18 to 2024.11.08
    * 500m x 500m, 8-Day composite
    * MODIS/061/MOD15A2H

5. Global Food-Support Analysis Data (GFSAD) Cropland
    * Global
    * 2010.01.01
    * 1000m x 1000m, one-time (2010)
    * EEID: USGS/GFSAD1000_V1

---
**Types of Vegetation Metrics**

- **Evapotranspiration (ET):** Measures the total water loss from soil and plants; higher ET indicates active plant growth and sufficient water availability. Over 25 years, an overall decrease in ET due to climate change could indicate reduced plant activity or increased water stress.

- **Fraction of Photosynthetically Active Radiation (FPAR):** Represents the fraction of sunlight absorbed by vegetation for photosynthesis; higher FPAR denotes healthier, more productive crops. Over 25 years, a declining FPAR trend could indicate an overall reduction in crop canopy density.

- **Leaf Area Index (LAI):** Quantifies the total leaf area per unit ground area; higher LAI reflects denser foliage and robust plant growth. Over 25 years, an overall decrease in LAI over time could point to diminished vegetation cover or stunted growth.

- **Enhanced Vegetation Index (EVI):** An index that enhances vegetation signals by reducing atmospheric and soil background noise; higher EVI values correspond to healthier vegetation. Over 25 years, a downward trend in EVI might indicate declining vegetation vigor.

- **Normalized Difference Vegetation Index (NDVI):** Assesses vegetation greenness by comparing red and near-infrared reflectance; higher NDVI values signify healthier, greener vegetation. Over 25 years, a decrease in NDVI over 25 years could suggest reduced vegetation health or coverage.

**NDVI and EVI** capture vegetation greenness and vigor, offering a direct assessment of plant health, whereas **LAI**, **FPAR**, and **ET** provide information on structural and functional aspects of vegetation. The scientific basis for preferring NDVI/EVI lies in their sensitivity to chlorophyll content and ability to detect changes in vegetation health more directly. The general hypothesis is: if the climate in 2000 was more favorable for crops than in 2024, we would expect to see higher values of ET, FPAR, LAI, EVI, and NDVI in 2000, reflecting more vigorous plant growth, denser foliage, and healthier vegetation due to optimal growing conditions. 

In [1]:
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
# Given an image file from GEE, count pixels in full image
def pixel_count(map_img, scale=None, geometry=None):

  # Set the default geometry to cover the entire image;
  # otherwise, use supplied geometry
  if geometry is None:
    geometry = map_img.geometry()

  # Set default scale to the image's native scale;
  # otherwise, use supplied scale
  if scale is None:
      scale = map_img.projection().nominalScale().getInfo()

  count = map_img.reduceRegion(
    reducer=ee.Reducer.count(),   # Aggregation function: Counts non-masked pixels
    geometry=geometry,            # Region of interest (default or user-supplied)
    scale=scale,                  # Spatial resolution for the operation 
    maxPixels=1e13                # Maximum number of pixels to process
  ).getInfo()

  return count

In [3]:
##### STANDARD PARAMETERS #######
# Define ROI (contiguous United States)
roi = ee.Geometry.Rectangle([-125, 24, -65, 50])

# Define time range
start_date = '2000-02-18'
end_date = '2023-12-31'

In [4]:
# Get date from image
def get_date(image):
  return ee.Image(image).date().format('YYYY-MM-dd')

# Base function for applying North America-centric projection specification to a specifc img instance
def set_std_NA_projection(daymet_img):

    # Define the DAYMET projection using ee.Projection: NAD83 / Canada Lambert (EPSG:3347)
    NA_projection = ee.Projection('EPSG:3347') 

    return daymet_img.reproject(
        crs=NA_projection,
        scale=daymet_img.projection().nominalScale()    # Use Daymet's native resolution
    )

# Define a function to calculate weekly averages
def weekly_average_daymet(date, daily_collection, start_date='2000-02-18'):
    start = ee.Date(date)
    end = start.advance(1, 'week')
    filtered_collection = daily_collection.filterDate(start, end)
    mean_image = filtered_collection.mean()

    original_crs = daily_collection.first().projection().crs()
    original_scale = daily_collection.first().projection().nominalScale()
    mean_image = mean_image.setDefaultProjection(crs=original_crs, scale=original_scale)

    # Calculate week number since the start date
    week_number = start.difference(ee.Date(start_date), 'week').add(1)
    
    mean_image = mean_image.set('system:time_start', start.millis())
    mean_image = mean_image.set('week', week_number)

    return mean_image

### DAYMETv4 pre-processing
# Load and filter Daymet data; set default projection
daymet_daily = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)
daymet_daily = daymet_daily.map(set_std_NA_projection)

average_map = lambda date:  weekly_average_daymet(date, daily_collection=daymet_daily)

# Create a list of starting dates for each week
start_dates = ee.List.sequence(
    ee.Date(start_date).millis(),
    ee.Date(end_date).millis(),
    7 * 24 * 60 * 60 * 1000  # 7 days in milliseconds
)

# Map the weekly_average function over the list of start dates
weekly_averages = start_dates.map(average_map)

# Convert the list of weekly averages to an ImageCollection
daymet_weekly = ee.ImageCollection(weekly_averages)
daymet_weekly = daymet_weekly.map(set_std_NA_projection)

# Status Check
print('Number of DAYMETv4 weekly images:', daymet_weekly.size().getInfo())
print(daymet_weekly.first().propertyNames().getInfo())
print(daymet_weekly.first().bandNames().getInfo())
print(daymet_weekly.first().get('week').getInfo())
print(f"Spatial resolution of the weekly image: {daymet_weekly.first().projection().nominalScale().getInfo()} meters")
print(f"CRS: {daymet_weekly.first().projection().crs().getInfo()}")

# Print the list of dates
weekly_dates_list = daymet_weekly.toList(daymet_weekly.size()).map(get_date)
print('Weekly Dates:', weekly_dates_list.getInfo()) 

Number of DAYMETv4 weekly images: 1246
['system:time_start', 'week', 'system:index', 'system:bands', 'system:band_names']
['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
1
Spatial resolution of the weekly image: 1000 meters
CRS: EPSG:3347
Weekly Dates: ['2000-02-18', '2000-02-25', '2000-03-03', '2000-03-10', '2000-03-17', '2000-03-24', '2000-03-31', '2000-04-07', '2000-04-14', '2000-04-21', '2000-04-28', '2000-05-05', '2000-05-12', '2000-05-19', '2000-05-26', '2000-06-02', '2000-06-09', '2000-06-16', '2000-06-23', '2000-06-30', '2000-07-07', '2000-07-14', '2000-07-21', '2000-07-28', '2000-08-04', '2000-08-11', '2000-08-18', '2000-08-25', '2000-09-01', '2000-09-08', '2000-09-15', '2000-09-22', '2000-09-29', '2000-10-06', '2000-10-13', '2000-10-20', '2000-10-27', '2000-11-03', '2000-11-10', '2000-11-17', '2000-11-24', '2000-12-01', '2000-12-08', '2000-12-15', '2000-12-22', '2000-12-29', '2001-01-05', '2001-01-12', '2001-01-19', '2001-01-26', '2001-02-02', '2001-02-09', '2001-02-16'

In [5]:
def weekly_interpolation(img_collection, start_date, end_date):
  """Interpolates an ImageCollection to weekly averages."""

  # Create a list of weekly start dates
  weekly_dates = ee.List.sequence(
      ee.Date(start_date).millis(),  # Use .millis() directly
      ee.Date(end_date).millis(),    # Use .millis() directly
      7 * 24 * 60 * 60 * 1000  # 7 days in milliseconds
  )

  # Function to compute weekly average
  def weekly_average(date_millis):
    date = ee.Date(date_millis)
    start = date.advance(-8, 'day')  # 8 days before
    end = date.advance(8, 'day')     # 8 days after

    # use 17-day window to ensure at least one data point is captured, avoiding null bands 
    filtered = img_collection.filterDate(start, end)

    original_crs = img_collection.first().projection().crs()
    original_scale = img_collection.first().projection().nominalScale()

    weekly_avg = filtered.mean().set('system:time_start', date_millis)
    weekly_avg = weekly_avg.setDefaultProjection(crs=original_crs, scale=original_scale)

    return weekly_avg

  # Map the weekly average function over the date list
  weekly_evi = weekly_dates.map(weekly_average)

  return ee.ImageCollection(weekly_evi)

### MODIS EVI
# Load and filter MODIS-EVI data
modis_EVI = ee.ImageCollection('MODIS/061/MOD13A2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select(['EVI', 'NDVI'])

# Interpolate data to produce weekly images
weekly_EVI = weekly_interpolation(modis_EVI, start_date, end_date)

# Reproject to 1000m x 1000m while keeping the original projection
weekly_EVI = weekly_EVI.map(lambda image: image.reproject(
    crs=image.projection(),  # Maintain the original projection
    scale=1000               # 1km resolution
))

### Status Check
print('Number of Weekly MODIS-EVI images:', weekly_EVI.size().getInfo())
print(weekly_EVI.first().bandNames().getInfo())
print(f"Spatial resolution of weekly_EVI image: {weekly_EVI.first().projection().nominalScale().getInfo()} meters")
print(f"CRS: {weekly_EVI.first().projection().crs().getInfo()}")
print()

Number of Weekly MODIS-EVI images: 1246
['EVI', 'NDVI']
Spatial resolution of weekly_EVI image: 1000 meters
CRS: SR-ORG:6974



In [6]:
### MODIS ET (500m)
# Load and filter MODIS-ET data
modis_ET = ee.ImageCollection('MODIS/061/MOD16A2GF') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select(['ET', 'LE', 'PET', 'PLE'])

# Downsample MODIS ET to 1km
modis_ET = modis_ET.map(lambda image: image.reduceResolution(
    reducer=ee.Reducer.mean(),
    maxPixels=1024  # This is important to avoid exceeding memory limits
).reproject(
    crs=image.projection(),  # Maintain the original projection
    scale=1000               # 1km resolution
))

# Interpolate data to produce weekly images
weekly_ET = weekly_interpolation(modis_ET, start_date, end_date)

### MODIS FPAR (500m)
# Load and filter MODIS-FPAR data
modis_FPAR = ee.ImageCollection('MODIS/061/MOD15A2H') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select(['Fpar_500m', 'Lai_500m'],  # Old band names
            ['FPAR', 'LAI'],            # New band names
    )

# Downsample MODIS FPAR to 1km
modis_FPAR = modis_FPAR.map(lambda image: image.reduceResolution(
    reducer=ee.Reducer.mean(),
    maxPixels=1024
).reproject(
    crs=image.projection(),
    scale=1000
))

# Interpolate data to produce weekly images
weekly_FPAR = weekly_interpolation(modis_FPAR, start_date, end_date)

##### status check
print('Number of Weekly MODIS-ET images:', weekly_EVI.size().getInfo())
print(weekly_ET.first().bandNames().getInfo())
print(f"Spatial resolution of the image: {weekly_ET.first().projection().nominalScale().getInfo()} meters")
print(f"CRS: {weekly_ET.first().projection().crs().getInfo()}")
print()

print('Number of Weekly MODIS-FPAR images:', weekly_EVI.size().getInfo())
print(weekly_FPAR.first().bandNames().getInfo())
print(f"Spatial resolution of the image: {weekly_FPAR.first().projection().nominalScale().getInfo()} meters")
print(f"CRS: {weekly_FPAR.first().projection().crs().getInfo()}")
print()

Number of Weekly MODIS-ET images: 1246
['ET', 'LE', 'PET', 'PLE']
Spatial resolution of the image: 1000.0000000000001 meters
CRS: SR-ORG:6974

Number of Weekly MODIS-FPAR images: 1246
['FPAR', 'LAI']
Spatial resolution of the image: 1000.0000000000001 meters
CRS: SR-ORG:6974



In [7]:
### GFSAD
# Define a function to reclassify the land cover values to:
# (0) no crops
# (1) irrigated crops
# (2) rainfed crops
def reclassify_gfsad_landcover(image):
  # Create a dictionary mapping old values to new values
  remap_values = {
      0: 0,  # Non-croplands remain 0
      1: 1,  # Irrigation major becomes irrigated (1)
      2: 1,  # Irrigation minor becomes irrigated (1)
      3: 2,  # Rainfed becomes rainfed (2)
      4: 2,  # Rainfed with minor fragments becomes rainfed (2) 
      5: 2,  # Rainfed with very minor fragments becomes rainfed (2)
  }

  # Use the remap() function to reclassify the image
  reclassified = image.remap(
      list(remap_values.keys()), list(remap_values.values())
  )

  reclassified = reclassified.rename(['crop'])

  return reclassified

# Load GFSAD data and reclassify input data
gfsad = reclassify_gfsad_landcover(ee.Image('USGS/GFSAD1000_V1'))

# status check
print(gfsad.bandNames().getInfo())
print(f"Spatial resolution of the GFSAD image: {gfsad.projection().nominalScale().getInfo()} meters")
print(f"CRS of GFSAD: {gfsad.projection().crs().getInfo()}")

['crop']
Spatial resolution of the GFSAD image: 993.882349974298 meters
CRS of GFSAD: EPSG:4326


In [8]:
datasets = [daymet_weekly, weekly_EVI, weekly_ET, weekly_FPAR]
limit = 1245
limited_datasets = [col.limit(limit) for col in datasets]

# Combine the limited collections
merged_dataset = ee.ImageCollection(limited_datasets[0])
for img_set in limited_datasets[1:]:
  merged_dataset = merged_dataset.combine(img_set)

# Add gfsad band data to all images in the merged datasets
band_order = ['crop', 'dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp', 'EVI', 'NDVI', 'ET', 'LE', 'PET', 'PLE', 'FPAR', 'LAI']
add_gfsad_bands = lambda img: img.addBands(gfsad)
reorder_bands = lambda img: img.select(band_order)
merged_dataset = merged_dataset.map(add_gfsad_bands)

# Skip week 71, reorder all bands; week 71 is missing FPAR data....
merged_dataset = merged_dataset.filter(ee.Filter.neq('system:index', '70')).map(reorder_bands)

print(merged_dataset.size().getInfo())
print(merged_dataset.first().bandNames().getInfo())

1244
['crop', 'dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp', 'EVI', 'NDVI', 'ET', 'LE', 'PET', 'PLE', 'FPAR', 'LAI']


In [9]:
# Ensure all images across all bands have consistent, common projection
# Since focusing on CONUS, use EPSG:3347 as default
target_projection_crs = ee.Projection('EPSG:3347') 
target_scale = 1000  # meters

def reproject_image(image):
    return image.reproject(crs=target_projection_crs, scale=target_scale)

merged_dataset = merged_dataset.map(reproject_image)

print(f"Spatial resolution of merged data image: {merged_dataset.first().projection().nominalScale().getInfo()} meters")
print(f"CRS: {merged_dataset.first().projection().crs().getInfo()}")

Spatial resolution of merged data image: 1000 meters
CRS: EPSG:3347


In [62]:
import time
ee.Authenticate()
ee.Initialize()

# Load the US states FeatureCollection
states = ee.FeatureCollection("TIGER/2018/States")

# Filter for California
roi = states.filter(ee.Filter.eq('NAME', 'Illinois'))

# Apply preprocessing to the ImageCollection
def process_and_export(image):
    
    # Preprocess the image (clip to ROI, resample, and convert bands)
    clipped_image = image.clip(roi).toFloat().resample('bilinear')
    
    # Get the date from the image 
    date = ee.Date(image.get('system:time_start'))
    date_string = date.format('YYYY-MM-dd').getInfo()

    # Calculate the week number
    start_date = ee.Date('2000-02-18') 
    week_number = date.difference(start_date, 'week').add(1).getInfo()

    # Pad the week number with zeros to ensure it has four digits
    week_string = str(week_number).zfill(4)

    # Create an export task for each image
    task = ee.batch.Export.image.toDrive(
        image=clipped_image,
        description=f'california_{date_string}',
        folder='EarthEngineExports',
        fileNamePrefix=f'illinois_1kmx1km_{week_string}_{date_string}',
        crs='EPSG:3347',
        scale=1000,
        region=roi.geometry().bounds(),
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    
    task.start()
    print(f"Started export task for {date_string}. Task ID: {task.id}")

    return(task)

# Iterate over all images in the ImageCollection
tasks = []
full_list = merged_dataset.toList(merged_dataset.size())
for i in range(merged_dataset.size().getInfo()):
    image = ee.Image(full_list.get(i))

    task = process_and_export(image)
    tasks.append((i, task))

    print(i + 1)
    print(task.status())
    print()

Started export task for 2000-02-18. Task ID: DIRRFM7FHRAKX4GDD3AXSLIR
1
{'state': 'READY', 'description': 'california_2000-02-18', 'priority': 100, 'creation_timestamp_ms': 1732675202342, 'update_timestamp_ms': 1732675202342, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DIRRFM7FHRAKX4GDD3AXSLIR', 'name': 'projects/project777-viirs-data/operations/DIRRFM7FHRAKX4GDD3AXSLIR'}

Started export task for 2000-02-25. Task ID: YZBMEU43UN4UKCRGZQ5ZBOQI
2
{'state': 'READY', 'description': 'california_2000-02-25', 'priority': 100, 'creation_timestamp_ms': 1732675223852, 'update_timestamp_ms': 1732675223852, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YZBMEU43UN4UKCRGZQ5ZBOQI', 'name': 'projects/project777-viirs-data/operations/YZBMEU43UN4UKCRGZQ5ZBOQI'}

Started export task for 2000-04-28. Task ID: TPSE6YBIA6GFT6PKGUQKDUAG
3
{'state': 'READY', 'description': 'california_2000-04-28', 'priority': 100, 'creation_timestamp_ms': 1732675244031, 'update_timestamp_ms': 173

KeyboardInterrupt: 

In [64]:
ee.batch.Export.imageCollection.toDrive()

AttributeError: type object 'Export' has no attribute 'imageCollection'

In [17]:
# Assuming your Earth Engine image is named 'image'
image = merged_dataset.first()
image = image.clip(roi).toFloat().resample('bilinear')

# Define the visualization parameters for each band
vis_params = {
    'crop': {
        'min': 0, 
        'max': 2, 
        'palette': ['gray', 'darkseagreen', 'darkgreen'] 
    },
    'dayl': {
        'min': 0, 
        'max': 86400, 
        'palette': ['midnightblue', 'navy', 'darkblue', 'royalblue', 'skyblue', 'gold', 'orange'] 
        # Transitioning from night to day with sunrise/sunset colors
    },
    'prcp': {
        'min': 0, 
        'max': 25, 
        'palette': ['white', 'powderblue', 'skyblue', 'deepskyblue', 'dodgerblue', 'blue', 'darkblue'] 
        # Increasing intensity of blue for higher precipitation
    },
    'srad': {
        'min': 0, 
        'max': 800, 
        'palette': ['black', 'dimgray', 'gray', 'lightgray', 'yellow', 'orange', 'red'] 
        # Gradual increase in brightness, ending with warm colors for high radiation
    },
    'swe': {
        'min': 0, 
        'max': 13931, 
        'palette': ['white', 'lightcyan', 'cyan', 'deepskyblue', 'dodgerblue', 'blue', 'navy'] 
        # Similar to precipitation, but with a cooler tone
    },
    'tmax': {
        'min': -40, 
        'max': 30, 
        'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'] 
    },
    'tmin': {
        'min': -60, 
        'max': 20, 
        'palette': ['darkblue', 'blue', 'lightblue', 'white', 'pink', 'orange', 'red'] 
        # Similar to tmax, but with a wider range and more gradual transition
    },
    'vp': {
        'min': 0, 
        'max': 8230, 
        'palette': ['white', 'aliceblue', 'lavender', 'lightskyblue', 'deepskyblue', 'royalblue', 'navy'] 
        # Light blues to represent vapor
    },
    'EVI': {
        'min': 2000, 
        'max': 8000, 
        'palette': ['red', 'brown', 'yellowgreen', 'green', 'darkgreen'] 
        # Representing unhealthy to healthy vegetation
    },
    'NDVI': { # Similar to EVI
        'min': 2000, 
        'max': 8000, 
        'palette': ['red', 'orange', 'yellowgreen', 'green', 'darkgreen'] 
    },
    'ET': {
        'min': 0, 
        'max': 200, 
        'palette': ['white', 'lightblue', 'skyblue', 'deepskyblue', 'blue', 'darkblue'] 
        # Similar to precipitation, as ET is related to water movement
    },
    'LE': {
        'min': -20, 
        'max': 10000, 
        'palette': ['white', 'lightpink', 'pink', 'hotpink', 'magenta', 'purple'] 
        # Using pinks and purples to differentiate from ET
    },
    'PET': {
        'min': -8, 
        'max': 793, 
        'palette': ['white', 'lightgoldenrodyellow', 'yellow', 'gold', 'orange', 'darkorange'] 
        # Yellows and oranges to represent potential evapotranspiration
    },
    'PLE': {
        'min': -40, 
        'max': 10000, 
        'palette': ['white', 'lightgreen', 'green', 'limegreen', 'forestgreen', 'darkgreen'] 
        # Greens to differentiate from LE
    },
    'FPAR': {
        'min': 0, 
        'max': 100, 
        'palette': ['brown', 'yellowgreen', 'green', 'darkgreen'] 
        # Representing absorption by green elements
    },
    'LAI': {
        'min': 0, 
        'max': 100, 
        'palette': ['tan', 'lightgreen', 'green', 'forestgreen', 'darkgreen'] 
        # Similar to FPAR, but with a lighter starting color
    },
}

# Define the center of the map (adjust to your needs)
map_center = [37.0902, -95.7129]  # Center of the continental US

# Create a Folium map
m = folium.Map(location=map_center, zoom_start=4)

# Add each band to the map as a layer
for band_name in vis_params:
    folium.TileLayer(
        tiles=image.select(band_name).getMapId(vis_params[band_name])['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=band_name,
        overlay=True,
    ).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('illinois.html')

In [ ]:
import folium
import ee

ee.Initialize()

# Define ROI (contiguous United States)
roi = ee.Geometry.Rectangle([-125, 24, -65, 50])

# Create a folium map
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add the bounding box to the map
folium.GeoJson(
    roi.getInfo(),
    name="Bounding Box",
    style_function=lambda x: {"color": "red", "weight": 2}
).add_to(m)

m

# Save and display the map
#m.save("roi_map.html")

In [54]:
import rasterio
import pandas as pd
from pyproj import Transformer

def geotiff_to_dataframe(geotiff_path):
    """
    Extracts data from a GeoTIFF file and converts it into a Pandas DataFrame.

    Args:
        geotiff_path (str): Path to the GeoTIFF file.

    Returns:
        pandas.DataFrame: DataFrame containing the extracted data.
    """

    with rasterio.open(geotiff_path) as src:
        # Read all bands as a NumPy array
        data = src.read()

        # Get latitude and longitude coordinates
        transformer = Transformer.from_crs(src.crs, "EPSG:4326")  # Transform from source CRS to WGS84
        rows, cols = data.shape[1:]
        lats, longs = [], []
        for row in range(rows):
            for col in range(cols):
                x, y = src.xy(row, col)
                latitude, longitude = transformer.transform(x, y)
                lats.append(latitude)
                longs.append(longitude)

        # Create a Pandas DataFrame
        df = pd.DataFrame({
            'lat': lats,
            'long': longs,
            'crop': data[0].flatten(),
            'dayl': data[1].flatten(),
            'prcp': data[2].flatten(),
            'srad': data[3].flatten(),
            'swe': data[4].flatten(),
            'tmax': data[5].flatten(),
            'tmin': data[6].flatten(),
            'vp': data[7].flatten(),
            'EVI': data[8].flatten(),
            'NDVI': data[9].flatten(),
            'ET': data[10].flatten(),
            'LE': data[11].flatten(),
            'PET': data[12].flatten(),
            'PLE': data[13].flatten(),
            'FPAR': data[14].flatten(),
            'LAI': data[15].flatten(),
        })

    return df

# Example usage
geotiff_file = '/home/katavga/Downloads/illinois_1kmx1km_2002-01-18.tif'  # Replace with your GeoTIFF file path
df = geotiff_to_dataframe(geotiff_file)
print(df)

              lat       long  crop  dayl  prcp  srad  swe  tmax  tmin  vp  \
0       42.641711 -91.517294   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
1       42.641662 -91.505451   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
2       42.641611 -91.493608   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
3       42.641559 -91.481766   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
4       42.641506 -91.469923   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
...           ...        ...   ...   ...   ...   ...  ...   ...   ...  ..   
284455  36.824745 -87.070765   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
284456  36.824107 -87.060276   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
284457  36.823468 -87.049788   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
284458  36.822828 -87.039299   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   
284459  36.822186 -87.028811   NaN   NaN   NaN   NaN  NaN   NaN   NaN NaN   

        EVI  NDVI  ET  LE  PET  PLE  FPAR  LAI  
0       NaN   NaN NaN NaN 